<a href="https://colab.research.google.com/github/willywildan/Analisis-Sentimen-SVM/blob/main/Sentiment_analysis_of_skincare_product_reviews_using_support_vector_machine_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
pip install sklearn

In [45]:
pip install Sastrawi

Import library

In [46]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn import metrics
nltk.download('punkt')
import string
import pickle
import matplotlib.pyplot as plt
#import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize, MWETokenizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
#Filtering (stopword Removal)
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
#Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
data = pd.read_csv("/content/drive/My Drive/Project/Thesis/reviewFD.csv", sep= ',')
data.head()

,title,review,kategori
0,NIVEA Creme,kalau beli nivea cream ini dimana ya kayaknya ...,negatif
1,NIVEA Creme,iya kayaknya emang enggak cocok soalnya semenj...,negatif
2,NIVEA Creme,NIVEA creme bisa bantu melembapkan bibir dan b...,positif
3,NIVEA Creme,Senang banget sama nivea creme inih...lembab b...,positif
4,NIVEA Creme,@hidden Content wah syukur deh cocok jg dipake...,positif


Data Cleaning

In [49]:
data.shape

(1259, 3)

In [50]:
data.duplicated().sum()

6

In [51]:
data.isna().sum()

title       0
review      0
kategori    0
dtype: int64

In [52]:
data.dropna().sum()

title       NIVEA CremeNIVEA CremeNIVEA CremeNIVEA CremeNI...
review      kalau beli nivea cream ini dimana ya kayaknya ...
kategori    negatifnegatifpositifpositifpositifnegatifposi...
dtype: object

In [53]:
def regex(text):
  #Removing mention & hashtag
  username = re.sub(r"@(\w+)"," ", text, flags = re.I)
  hashtag = re.sub(r"#(\w+)"," ", username, flags = re.I)
  #Removing characters
  #chara = re.sub(r"[<,`+~'!^&?\".=@*/$%_:;()->]"," ", hashtag, flags=re.I)
  tanda = re.sub (r"\W", " ",hashtag, flags=re.I )
  number = re.sub (r"\d", " ",tanda, flags=re.I )
  #removing single character
  #single = re.sub(r"\s+[a-zA-Z]\s+"," ", chara, flags = re.I)
  single = re.sub(r"\b[a-zA-Z]\b"," ", number, flags = re.I)
  #Substituting multiple spaces with single space
  spaces = re.sub(r"\s+"," ", single, flags = re.I)
  # --- Case Folding ---
  text = spaces.lower()
  return text

In [54]:
data['review'] = data['review'].apply(regex)
data.head()

,title,review,kategori
0,NIVEA Creme,kalau beli nivea cream ini dimana ya kayaknya ...,negatif
1,NIVEA Creme,iya kayaknya emang enggak cocok soalnya semenj...,negatif
2,NIVEA Creme,nivea creme bisa bantu melembapkan bibir dan b...,positif
3,NIVEA Creme,senang banget sama nivea creme inih lembab bgt...,positif
4,NIVEA Creme,content wah syukur deh cocok jg dipake masker...,positif


In [55]:
re_stopword = pd.DataFrame(data['review'])
re_stopword.to_csv("/content/drive/My Drive/Project/Thesis/result/casefolding.csv", sep= ',')

Formalize

In [56]:
#colloquial loading
def get_lookup():
  df = pd.read_csv("/content/drive/My Drive/Project/Thesis/dictionary.csv")
  df_record = df.to_dict('records')
  return {  entry['slang']: entry['formal'] for entry in df_record}

In [57]:
lookup = get_lookup()

In [58]:
def formalize(text):
  text = nltk.tokenize.word_tokenize(text)
  #print(text)
  formalized = [lookup[token] if token in lookup else token for token in text]
   #print(lookup) 
  if formalized:
    re_tokens = formalized
  else:
    re_tokens = text
  freq_tokens = nltk.FreqDist(formalized)

  #mword = tk.tokenize(freq_tokens)
  return " ".join(freq_tokens)

In [59]:
data['review_formalize'] = data['review'].apply(formalize)
data.head()

,title,review,kategori,review_formalize
0,NIVEA Creme,kalau beli nivea cream ini dimana ya kayaknya ...,negatif,kalau beli nivea krim ini di mana ya sepertiny...
1,NIVEA Creme,iya kayaknya emang enggak cocok soalnya semenj...,negatif,iya sepertinya memang enggak cocok soalnya sem...
2,NIVEA Creme,nivea creme bisa bantu melembapkan bibir dan b...,positif,nivea creme bisa bantu melembapkan bibir dan b...
3,NIVEA Creme,senang banget sama nivea creme inih lembab bgt...,positif,senang banget sama nivea creme ini lembap buat...
4,NIVEA Creme,content wah syukur deh cocok jg dipake masker...,positif,content wah syukur deh cocok juga dipakai mask...


In [60]:
re_stopword = pd.DataFrame(data['review_formalize'])
re_stopword.to_csv("/content/drive/My Drive/Project/Thesis/result/formalize.csv", sep= ',')
#re_stopword.get_features

First Tokenizing

In [61]:
def tokenize(text):
  tk = MWETokenizer([('enggak','ada','palsunya'),('enggak','ada','efek','sampingnya'),('enggak','ada','efek','jelek'),('enggak','perlu','kerja','tambahan'),('enggak','perlu','di','ragukan'),('enggak','ada','reaksi','negatif'),
                     ('bekerja','keras','mengeluarkan','kotoran'),('menghilangkan','jerawat','minyak'),('komedo','hitam'),('komedo','putih'),('semua','kotoran'),('mengangkat','kotoran'),('enggak','ada','yang','cocok'),
                     ('kadaluwarsa','enggak','ada'),('pohon','minyak'),('engga','kecewa'),('tidak','mengecewakan'),('tidak','memperbaiki','tekstur'),('tidak','hilang'),('komedo','menghilang'),('mengurangi','tambang','minyak'),('jerawat','enggak','tumbuh'),('tipe','kulit','berminyak'),

                     ('enggak','terlihat','jerawatnya'),('enggak','begitu','ampuh'),('enggak','ada','lengket'),('enggak','ada','perubahan'),('belum','ada','perubahan'),('enggak','buat','breakout'),('enggak','bikin','breakout'),('enggak','pernah','separah'),('enggak','memakai','sunscreen'),
                     ('enggak','terlalu','berminyak'),('enggak','akan','sesakit'),('enggak','akan','sakit'),('enggak','begitu','kering'),('enggak','begitu','berminyak'),('tanpa','bikin','breakout'),('enggak','pernah','breakout'),('enggak','pernah','jerawatan'),
                     ('enggak','jadi','tambang','minyak'),('enggak','jadi','jerawat'),('enggak','sempat','regenerasi'),('enggak','lagi','bermasalah'),('belum','ada','efek'),
                     ('enggak','perlu','diragukan'),('enggak','ada','kerutan'),('enggak','ingin','rugi'),('enggak','berasa','gerah'),('enggak','bikin','beruntusan'),('enggak','bikin','keset'),
                     ('enggak','kelihatan'),('enggak','terlihat'),('enggak','ada','gatal'),('enggak','ingin','keluar'),('enggak','terlalu','susah'),
                     ('enggak','bisa','menghilangkan'),('enggak','ada','masalah'),('enggak','terlalu','minyakan'),('ke','punggung','enggak'),('jerawatan','juga','enggak'),
                     ('sudah','lama','enggak'),('enggak','bisa','berhenti'),('enggak','begitu','dianjurkan'),('enggak','ada','jerawat'),
                     ('enggak','terlalu','beda'),('enggak','cuma','wajah'),('enggak','dua','kali'),('enggak','terlalu','kering'),('enggak','pernah','cocok'),
                     ('enggak','ketergantungan'),('enggak','ingin','pakai'),('enggak','terlalu','berefek'),('enggak','begitu','ampuh'),('enggak','bisa','hilang'),
                     ('beruntus','enggak','berisi'),('enggak','berrasa','lengket'),('beruntus','enggak','hilang'),('tidak','ada','perubahan'),
                     ('tidak','mengandung','merkuri'),('tidak','keluar','jerawat'),('jerawat','cepat','hilang'),('kurang','percaya','diri'),('percaya','diri'),
                     ('enggak','hilang'),('dasar','kulit','kebal'),('rambut','rontok','berkurang'),('ada','bekas','jerawat'),
                     ('jerawat','jadi','hilang'),('jerawat','susah','hilang'),('jerawat','pada','keluar'),('mengurangi','kerut'),('enggak','sembuh'),
                     ('jerawat','pada','kering'),('belum','hilang'),('belum','menghilang'),('jerawatku','sudah','pudar'),('minyak','menghilang'),
                     ('jarang','muncul'),('kempis','kering'),('jerawat','jadi','gede'),('enggak','meresap','sempurna'),('musim','dingin'),('tidak','murah'),
                     ('komedo','lumayan','bersih'),('malu','buat','perawatan'),('ketombe','tambah','banyak'),('mengurangi','rambut','rontok'),('tahan','air'),
                     ('mudah','dipalsu'),('jerawat','cepat','kempis'),('belum','terlalu','berefek'),('luar','biasa','kering'),('jerawat','masih','ada'),('enggak','ada','alkoholnya'),                     

                     ('tidak','sehitam'),('enggak','manjur'),('enggak','menyedot'),('enggak','lengket'),('enggak','suka'),('enggak','jelas'),('enggak','jodoh'),('enggak','ganggu'),('enggak','merasakan'),('enggak','kecium'),
                     ('enggak','kering'),('enggak','pahit'),('enggak','berminyak'),('enggak','berani'),('enggak','belang'),('enggak','lanjut'),('enggak','banget'),('enggak','gatal'),
                      ('enggak','hilang'),('enggak','iritasi'),('enggak','jual'),('tahu','enggak'),('enggak','sukanya'),('enggak','bisa'),('enggak','kombinasi'),('enggak','mengefek'),
                     ('enggak','meninggalkan'),('enggak','alkohol'),('enggak','perih'),('enggak','sadar'),('enggak','tahu'),('enggak','ingin'),('enggak','rontok'),('enggak','apa'),
                     ('enggak','berefek'),('enggak','cocok'),('enggak','sembuh'),('enggak','gumpal'),('enggak','sabar'),('enggak','datang'),('enggak','mengecewakan'),('kurang','cocok'),
                     ('enggak','ada'),('enggak','jerawatan'),('enggak','salah'),('enggak','berlebihan'),('enggak','beranjak'),('enggak','nyaman'),('kurang','efektif'),
                     ('enggak','diragukan'),('enggak','berbahaya'),('enggak','minyakan'),('enggak','menyengat'),('enggak','semahal'),('enggak','boleh'),('enggak','kusam'),
                     ('enggak','banjir'),('enggak','terawat'),('enggak','pernah'),('enggak','rata'),('enggak','bermasalah'),('enggak','bagus'),('enggak','sekeras'),('agak','kurang'),
                     ('enggak','lagi'),('enggak','masalah'),('enggak','berdenyut'),('takut','kering'),('enggak','mempengaruhi'),('enggak','repot'),('enggak','breakout'),
                     ('enggak','lama'),('enggak','berbulir'),('enggak','dipakai'),('enggak','karuan'),('enggak','apa-apa'),('enggak','berrasa'),('enggak','juga'),('banyak','jerawat'),
                     ('enggak','bergumpal'),('enggak','lama'),('enggak','mahal'),('enggak','sakitnya'),('enggak','menyesal'),('enggak','habis'),('enggak','merah'),('enggak','ya'),
                     ('enggak','enak'),('enggak','mempan'),('enggak','bau'),('enggak','gatal'),('enggak','kasar'),('enggak','mengganggu'),('enggak','menyerap'),('kurang','lebih'),
                     ('enggak','cocoknya'),('enggak','menghitam'),('enggak','pakai'),('enggak','beruntusan'),('enggak','instan'),('boleh','enggak'),('enggak','kuat'),('enggak','menimbulkan'),
                     ('enggak','seekstrem'),('enggak','kerasa'),('enggak','tahan'),('timbul','jerawat'),('komedo','hilang'),('ingin','mencoba'),('mengandung','merkuri'),('kering','jerawat'),('banyak','komedo'),
                     ('enggak','pecah'),('kilang','minyak'),('jadi','beruntusan'),('jerawat','bernanah'),('jerawatpun','timbul'),('kurang','rajin'),('batal','mencari'),
                     ('tidak','boleh'),('tidak','merekomendasikan'),('tidak','mempengaruhi'),('membersihkan','susah'),('susah','membersihkan'),('jadi','jerawat'),('cukup','oke'),
                     ('non','logam'),('kurang','mantap'),('asli','hancur'),('komedo','berkurang'),('bikin','jerawatan'),('hilang','jerawatnya'),('kurang','nendang'),('beruntusan','hilang'),
                     ('kurang','nampol'),('bikin','breakout'),('cepat','berminyak'),('sangat','berminyak'),('bersih','porinya'),('kulit','berminyak'),('ingin','coba'),('kurang','suka'),
                     ('beruntusan','parah'),('cepat','kering'),('tumbuh','jerawat'),('tidak','berefek'),('malah','jerawatan'),('suka','banget'),('susah','mengangkat'),('kurang','lama'),('suka','berminyak'),
                     ('kurang','angkat'),('kontrol','minyak'),('makin','parah'),('makin','berminyak'),('cara','membersihkan'),('mengurangi','minyak'),('menghilangkan','minyak'),('berrasa','kering'),('menghilangkan','kusam'),
                     ('putih','tembok'),('jarang','jerawatan'),('nambah','jerawat'),('keluar','jerawat'),('rambut','rontok'),('kulit','cerah'),('jerawat','kalem'),('mengempiskan','jerawat'),('kusam','jerawatan'),
                     ('belum','kerasa'),('belum','berrasa'),('suka','tersiksa'),('jerawatan','banget'),('mengkeringkan','jerawat'),('menyembuhkan','breakout'),('terima','kasih'),('kempis','jerawat'),('kulit','kering'),('berkurang','minyaknya'),
                     ('berminyak','parah'),('jerawat','kabur'),('minyak','lenyap'),('kurang','minyak'),('oke','banget'),('jerawat','mengecil'),('jerawat','kering'),('jadi','jerawatan'),('mencegah','jerawat'),
                     ('lidah','buaya'),('lebih','susah'),('lebih','enak'),('enak','adem'),('yang','enak'),('habis','enak'),('enak','meratakannya'),('enak','banget'),('menghilangkan','jerawat'),('jerawat','tumbuh'),
                     ('menambah','jerawat'),('jerawat','kempis'),('jerawat','baru'),('kontrol','jerawat'),('cepat','keringnya'),('muncul','jerawat'),('kulit','sensitif'),('jerawat','hilang'),('belum','bersih'),
                     ('sari','ayu'),('berat','badan'),('beruntusan','jadinya'),('belum','pernah'),('kurang','enaknya'),('kurang','enak'),('flek','hitam'),('minyak','hilang'),('tambang','minyak'),('pohon','minyak'),('merah','muda'),
                     ('bebas','kerut'),('bebas','minyak'),('cuka','apel'),('berapa','lama'),('bekas','belang'),('sendok','pelastik'),('bau','busuk'),('bekas','jerawat'), ('bikin','jerawat'),])
  text = tk.tokenize(text.split())
  #print(text)
  return " ".join(text)

In [62]:
data['review_tokenize'] = data['review_formalize'].apply(tokenize)
data.head()

,title,review,kategori,review_formalize,review_tokenize
0,NIVEA Creme,kalau beli nivea cream ini dimana ya kayaknya ...,negatif,kalau beli nivea krim ini di mana ya sepertiny...,kalau beli nivea krim ini di mana ya sepertiny...
1,NIVEA Creme,iya kayaknya emang enggak cocok soalnya semenj...,negatif,iya sepertinya memang enggak cocok soalnya sem...,iya sepertinya memang enggak_cocok soalnya sem...
2,NIVEA Creme,nivea creme bisa bantu melembapkan bibir dan b...,positif,nivea creme bisa bantu melembapkan bibir dan b...,nivea creme bisa bantu melembapkan bibir dan b...
3,NIVEA Creme,senang banget sama nivea creme inih lembab bgt...,positif,senang banget sama nivea creme ini lembap buat...,senang banget sama nivea creme ini lembap buat...
4,NIVEA Creme,content wah syukur deh cocok jg dipake masker...,positif,content wah syukur deh cocok juga dipakai mask...,content wah syukur deh cocok juga dipakai mask...


In [63]:
re_stopword = pd.DataFrame(data['review_tokenize'])
re_stopword.to_csv("/content/drive/My Drive/Project/Thesis/result/tokenizing.csv", sep= ',')

In [64]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
newlist = ['semenjak','penuh','komedo_putih','komedo_hitam','bekas_jerawat','tercampur','campuran','siku','lutut','jenisnya','sejenis','kadaluwarsanya','dipalsukan','pekan','akhir','pelembap','berapa','berubah','terjadi','entah','sejenis','jenis','jenisnya','tipe','paling','waduh','aduh','pemakaian','apel','paling','dibawa',
           'kalinya','perasaan','makan','pemakaian','bahkan','bakal','menurutku','apapun','kata','cuka','masing','kondisi','plus','ditambah','kerasa','rasa','sinar','matahari','segala','kadang','tekstur','teksturnya','putih','minyaknya','flek_hitam','vitamin','sedang','hati','nafas','cairan','lipstik','besokannya','pengguna','termasuk','awalnya','pakainya',
           'dipakainya','mineral','semacam','macam','kuas','merah','berdenyut','denyutnya','tubuh','cari','dihargai','harga','kotak','ponds','dikulitku','makkah','angkatnya','daya','adanya','mengoles','diseluruh','seluruh','berbagi','kakak','ikutan','perdana','ibuku','pori','bilang','sewajah','hampir','sebulan','warnanya','badanku','kakinya',
           'dikaki','masa','kena','yogyakarta','bandung','jakarta','orang','mawar','adonannya','adonan','namanya','besok','besoknya','matanya','khasiatnya','sejauh','maksudnya','kemasannya','kemasan','produknya','daring','serbuknya','selesai','padahal','maskernya','maksimal','bersihnya','biar','terutama',
           'toko','beli','varian','merah_muda','lidah_buaya','biru','pokoknya','matanya','besoknya','cowok','komedo','jam','maskernya','pipi','dagu','hasil','kedua','sehabis','mandi','dijadikan','orang','walaupun','besoknya','matanya','punggung','sendiri','berarti','warna','ngomong-ngomong','barang','kontrolnya','bangun',
           'wanginya','baunya','sekali','kayaknya','semingguan','hidung','pengalaman','sering','selalu','berefek','tas','stok','lama','efeknya','jauh','cuci','cuka_apel','plastik','sendok','sumpah','maka','makanya','kerja','pulang','bertahun','tahun','berbahan','bahan','bahannya','hidung','dicampur','mencampur','campur',
           'mengadon','kelihatan','kelihatannya','menemukan','oles','wanginya','baca','sebelah', 'saudara','sih', 'tawa','akhirnya','nih','thread','hidden','content','kok','aku','kamu','mereka','banget','aztec','lebih','siang', 'saripohatji', 'kyusoku','bihaku','pakai','beruntus',
           'dicoba','punya','tambah','bibir','waktu','katanya','eyeliner','pas','ulasan','produk','masker','lah','halo','ah','perempuan','mam','bulan','malam','minggu','hari','bulanan','mingguan','mungkin','nya','nge','terlalu','bikin','lagi','creme','mak','dong','tuh','sendok_pelastik',
           'kalian','tanya','kasih','ulasannya','area','dandan','mata','kapas','melihatnya','kulitnya','teh','sejak','teman','temanku','informasi','bagikan','bagian','bekas_belang','badan','kaki','lihat','tangan','super','paginya','kulitmu','dikulit','kulitku','diwajah','wajah','jerawatnya',
           'jerawat','hasilnya','soalnya','jadi','mom','coba','ingin','selama','mulai','langsung','memang','bmc','krim','soal','satu','jadi','ibu','air','sini','semua','water','micellar','loreal','nivea','menjadi','ibu','terima_kasih',
           'semua_kotoran','guru','semalam','nah','sunblok','menit','tinggal','tempat','ternyata','merasa','berrasa','terlihat','habis','tetap','malah','dimukaku','pijat','bilas','tidur','tahu','baru','buat','jadinya','sekarang','pas','minum','sangat','wah','cuma','efek','tiap','terus','buat','kapsul','kulit','maskeran','mah','say','hai',
           'jepang','jerman','arab','tempel','menempel','wajahku','wajahnya','merek','menurutku','begini','memakai','tadi','dipakai','biasanya','apa','jerawatku','bedakan','sana','bawa','harganya','seminggu','beberapa','rasanya','banyak','semakin','makin','oke','bok','mencoba','sepertinya','bagaimana','sama','pernah','garnier','mbak','boks','suami',
           'musim_dingin','berat_badan','diberi','enggak_merasakan','merasakan','up','pohon','pohon_minyak','valent','sekolah','clay','heinz','tahesta','bragg','benaran','benar','cuka_apel','kuliah','si','yah','mana','kali','an', 'malam', 'bedak','pagi','kemarin','pertama','sapaan','sapa','iya','loh','kan','eh','senyum','sore','deh','kalau','enggak_banjir','enggak_gumpal','enggak_datang',
           'serum','toner','shiso','tbs','veet','rose','dewy','jar','oriflame','dokter','sari_ayu','mencuci','mencucinya','sa','gel','volcanic','jeju','wine','innisfree','hijau','telapak','ter','lalu','setelahnya','penjualnya','jual','daerah','ikut','serbuk','bentonite','kira','putihnya','idealnya','lipat','berasanya','didiamkan','alhasil','enggak_tahan','pencet','dahi','losion','ghassoul']
stopword.dictionary.words.remove('tidak')
stopword.dictionary.words.remove('agak')
stopword.dictionary.words.remove('nggak')
stopword.dictionary.words.remove('ingin')
stopword.dictionary.words.extend(newlist)
def stopwords(text):
  cleaned = stopword.remove(text)
  cleaned.shape
  return cleaned
#print(stopword.dictionary.words)

In [65]:
print('serum' in stopword.dictionary.words)

True


In [66]:
data['review_stop'] = data['review_tokenize'].apply(lambda text: " ".join(stopword.remove(text) for text in text.split()) )
data.head()

,title,review,kategori,review_formalize,review_tokenize,review_stop
0,NIVEA Creme,kalau beli nivea cream ini dimana ya kayaknya ...,negatif,kalau beli nivea krim ini di mana ya sepertiny...,kalau beli nivea krim ini di mana ya sepertiny...,agak susah
1,NIVEA Creme,iya kayaknya emang enggak cocok soalnya semenj...,negatif,iya sepertinya memang enggak cocok soalnya sem...,iya sepertinya memang enggak_cocok soalnya sem...,enggak_cocok kasar
2,NIVEA Creme,nivea creme bisa bantu melembapkan bibir dan b...,positif,nivea creme bisa bantu melembapkan bibir dan b...,nivea creme bisa bantu melembapkan bibir dan b...,bantu melembapkan pengelupasan mati ...
3,NIVEA Creme,senang banget sama nivea creme inih lembab bgt...,positif,senang banget sama nivea creme ini lembap buat...,senang banget sama nivea creme ini lembap buat...,senang lembap
4,NIVEA Creme,content wah syukur deh cocok jg dipake masker...,positif,content wah syukur deh cocok juga dipakai mask...,content wah syukur deh cocok juga dipakai mask...,syukur cocok lumayan alternatif ganti


In [67]:
re_stopword = pd.DataFrame(data['review_stop'])
re_stopword.to_csv("/content/drive/My Drive/Project/Thesis/result/stopword.csv", sep= ',')

In [68]:
#Stemming sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemming(text):
  list_tokens = text.split(' ')
  text = [(token + ":" + stemmer.stem(token)) for token in list_tokens]
  text = [stemmer.stem(token) for token in list_tokens]
  return " ".join(text)

In [69]:
data['review_stem'] = data['review_stop'].apply(stemming)
data.head()

,title,review,kategori,review_formalize,review_tokenize,review_stop,review_stem
0,NIVEA Creme,kalau beli nivea cream ini dimana ya kayaknya ...,negatif,kalau beli nivea krim ini di mana ya sepertiny...,kalau beli nivea krim ini di mana ya sepertiny...,agak susah,agak susah
1,NIVEA Creme,iya kayaknya emang enggak cocok soalnya semenj...,negatif,iya sepertinya memang enggak cocok soalnya sem...,iya sepertinya memang enggak_cocok soalnya sem...,enggak_cocok kasar,enggak cocok kasar
2,NIVEA Creme,nivea creme bisa bantu melembapkan bibir dan b...,positif,nivea creme bisa bantu melembapkan bibir dan b...,nivea creme bisa bantu melembapkan bibir dan b...,bantu melembapkan pengelupasan mati ...,bantu lembap kelupas mati mudah
3,NIVEA Creme,senang banget sama nivea creme inih lembab bgt...,positif,senang banget sama nivea creme ini lembap buat...,senang banget sama nivea creme ini lembap buat...,senang lembap,senang lembap
4,NIVEA Creme,content wah syukur deh cocok jg dipake masker...,positif,content wah syukur deh cocok juga dipakai mask...,content wah syukur deh cocok juga dipakai mask...,syukur cocok lumayan alternatif ganti,syukur cocok lumayan alternatif ganti


In [70]:
re_stemming = pd.DataFrame(data['review_stem'])
re_stemming.to_csv("/content/drive/My Drive/Project/Thesis/result/stemming.csv", sep= ',')

Fitur TF-IDF

In [71]:
# inisialisasi CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data['review_stem'].tolist())
X_train_counts.shape

(1259, 966)

In [72]:
check = count_vect.get_feature_names()
checkpd = pd.DataFrame(check)
checkpd.to_csv("/content/drive/My Drive/Project/Thesis/result/check.csv", sep= ',')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [73]:
count_vect.vocabulary_.get(u'algorithm')

In [74]:
#hitung nilai TFIDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#print(X_train_tfidf.get_feature_names())
print(X_train_tfidf.shape)

(1259, 966)


In [75]:
# print nilai idf 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=count_vect.get_feature_names(),columns=["idf_weights"])
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,idf_weights
enggak,2.122710
cocok,2.801329
jerawat,3.076272
cerah,3.411479
suka,3.429337
...,...
ngeletek,7.445720
ngeh,7.445720
fasial,7.445720
navaa,7.445720


In [76]:
#hitung skor TFIDF untuk ulasan
# count matrix 
count_vector=count_vect.transform(data['review_stem'].tolist()) 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)
feature_names = count_vect.get_feature_names() 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,tfidf
susah,0.779107
agak,0.626891
acung,0.000000
pelan,0.000000
parfume,0.000000
...,...
hore,0.000000
hutang,0.000000
ibukota,0.000000
icip,0.000000


In [77]:
# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
# just send in all your docs here
fitted_vectorizer=tfidf_vectorizer.fit(data['review_stem'].tolist())
tfidf_vectorizer_vectors=fitted_vectorizer.transform(data['review_stem'].tolist())
# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0] 
# place tf-idf values in a pandas data frame 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,tfidf
susah,0.779107
agak,0.626891
acung,0.000000
pelan,0.000000
parfume,0.000000
...,...
hore,0.000000
hutang,0.000000
ibukota,0.000000
icip,0.000000


Modelling - SVM

In [78]:
text_clfSVM = Pipeline([
                  ('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                   ('clf', svm.SVC(kernel='linear', gamma='scale'))])

In [79]:
X_train, X_test, y_train, y_test = train_test_split(data['review'],data['kategori'], test_size=0.2, random_state = 44)
model = text_clfSVM.fit(X_train, y_train)
y_score = model.predict(X_test)

print("\naccuracy :{}%".format(round(accuracy_score(y_test,y_score)*100,2)) )


accuracy :86.51%


In [80]:
re_stemming = pd.DataFrame(X_test)
re_stemming.to_csv("/content/drive/My Drive/Project/Thesis/result/datalatih.csv", sep= ',')

In [81]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

y_true = y_score
y_pred = y_test
target_names = ['negatif', 'netral', 'positif']
print("\naccuracy:{}".format(round(accuracy_score(y_true, y_pred)*100,2 )))
print("\nprecision:{}".format(round(precision_score(y_true, y_pred, average='macro')*100,2)))
print("\nrecall:{}".format(round(recall_score(y_true, y_pred, average='macro')*100,2)))

print(classification_report(y_true, y_pred, target_names=target_names))


accuracy:86.51

precision:81.01

recall:87.61
              precision    recall  f1-score   support

     negatif       0.87      0.85      0.86        95
      netral       0.65      0.91      0.75        22
     positif       0.91      0.87      0.89       135

    accuracy                           0.87       252
   macro avg       0.81      0.88      0.84       252
weighted avg       0.87      0.87      0.87       252



Test & Save Model

In [82]:
i = pd.Series(["engga ada perubahan dikulit"])
model.predict(i)

array(['negatif'], dtype=object)

In [83]:
i = pd.Series(["produk ini bagus banget bisa mencerahkan kulit"])
model.predict(i)

array(['positif'], dtype=object)

In [84]:
with open("/content/drive/My Drive/Project/Thesis/result/model.pkl", 'wb') as file:
    pickle.dump(model, file)